In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D
import pandas as pd
from preprocessing import load_and_preprocess_data

from sklearn.model_selection import train_test_split

In [2]:
padded_docs, artists_onehot_encoded, vocab_size, max_length = load_and_preprocess_data("spotify_millsongdata.csv")

In [3]:
X_train, X_test, y_train, y_test = train_test_split(padded_docs, artists_onehot_encoded, test_size=0.2, random_state=42)

In [25]:
padded_docs[0].shape

(950,)

In [4]:
# lstm = keras.layers.LSTM(units=artists_onehot_encoded.shape[1])
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=vocab_size, output_dim = 128, input_length = padded_docs[0].shape[0]))
model_lstm.add(SpatialDropout1D(0.3))
model_lstm.add(LSTM(256, dropout = 0.3, recurrent_dropout = 0.3))
model_lstm.add(Dense(256, activation = 'relu'))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(artists_onehot_encoded.shape[1], activation = 'softmax'))
model_lstm.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

In [5]:
history = model_lstm.fit(
    X_train,
    y_train,
    validation_split = 0.1,
    epochs = 8,
    batch_size = 512
)


Epoch 1/8
62/62 [==============================] - 80678s 1321s/step - loss: 6.2104 - accuracy: 0.0036 - val_loss: 6.1794 - val_accuracy: 0.0026
Epoch 2/8


In [ ]:
def load_and_preprocess_data(path, in_colab=False):
    """
    Load the data and preprocess it, expect runtime of 20 seconds.
    :param path: path to the data
    :return: preprocessed data in the form of a pandas dataframe. The first item returned is the data,
    the second is the labels, the third is the vocabulary size, and the fourth is the maximum length of a sequence
    """
    df = pd.read_csv(path)
      
    # remove artist with fewer than 30 songs
    df = df.groupby('artist').filter(lambda x: len(x) > 30)

    df['text'] = df['text'].apply(preprocess)

    # Identify the rows that contain duplicated text in the 'song' column
    no_covers = ~df['song'].duplicated()

    # Filter the DataFrame to include only the rows with unique text
    df = df[no_covers]

    # prepare text data for a recurrent network
    return encode_text_and_labels(df)

In [3]:
df = pd.read_csv("spotify_millsongdata.csv")
    
# remove artist with fewer than 30 songs
df = df.groupby('artist').filter(lambda x: len(x) > 100)

In [5]:
df['artist'].value_counts()

Donna Summer            191
Gordon Lightfoot        189
Bob Dylan               188
George Strait           188
Cher                    187
                       ... 
Gloria Estefan          102
Wishbone Ash            102
Alan Parsons Project    102
Jason Mraz              101
Regine Velasquez        101
Name: artist, Length: 268, dtype: int64